In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
# !pip install google-meridian[colab,and-cuda]==1.0.3

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git"

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

In [ ]:
import pandas as pd
from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter
import io
import requests


coord_to_columns = load.CoordToColumns(
    time='Date',
    #geo='geo',
    controls=['Organic_impressions'],
    #population='population',
    kpi='total_revenue',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'bing_impressions',
        'google_impressions',
        'tiktok_impressions',
        'pinterest_impressions',
    ],
    media_spend=[
       'bing_spend',
      'pinterest_spend',
      'facebook_spend',
      'google_spend',
      'tiktok_spend',
    ],
    #organic_media=['Organic_impressions'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'bing_impressions': 'Bing',
    'google_impressions': 'Google',
    'tiktok_impressions': 'Tiktok',
    'pinterest_impressions': 'Pinterest',
}
correct_media_spend_to_channel = {
    'bing_spend': 'Bing',
    'pinterest_spend': 'Pinterest',
    'facebook_spend': 'Facebook',
    'google_spend': 'Google',
    'tiktok_spend': 'Tiktok'
}
# Download the CSV data
url = 'https://raw.githubusercontent.com/Connective3Innovation/impact_V2/main/Data/charles%26ivy_apr2023_apr_2025_cleaned.csv' #Modified URL to raw content
response = requests.get(url)
response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
csv_data = response.content.decode('utf-8') #Assuming UTF-8 encoding

# Load into pandas DataFrame
df = pd.read_csv(io.StringIO(csv_data), sep=',') #Explicitly setting separator

loader = load.DataFrameDataLoader(
    #csv_path='https://github.com/Connective3Innovation/impact_V2/blob/main/Data/charles%26ivy_apr2023_apr_2025_cleaned.csv',
    df=df, # Pass the DataFrame directly
    kpi_type='revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

In [ ]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1)

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive'
start_date = '2023-03-27'
end_date = '2025-04-28' # Changed end_date to a valid date
mmm_summarizer.output_model_results_summary('summary_output_charlesivy_v1.html', filepath, start_date, end_date)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/summary_output_charlesivy_v1.html')